# Numeric and Categorical Value Analysis

In [2]:
import requests
import pandas
#import numpy as np
url = "https://raw.githubusercontent.com/tulip-lab/sit742/master/Assessment/2019/data/wine.json"
r = requests.get(url, headers={'Accept': 'application/json'})
data = pandas.DataFrame(r.json())
print(data.head())

    country                                        description  \
0     Italy  Aromas include tropical fruit, broom, brimston...   
1  Portugal  This is ripe and fruity, a wine that is smooth...   
2        US  Tart and snappy, the flavors of lime flesh and...   
3        US  Pineapple rind, lemon pith and orange blossom ...   
4        US  Much like the regular bottling from 2012, this...   

                          designation points  price           province  \
0                        Vulkà Bianco     87    NaN  Sicily & Sardinia   
1                            Avidagos     87   15.0              Douro   
2                                None     87   14.0             Oregon   
3                Reserve Late Harvest     87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block     87   65.0             Oregon   

              region_1           region_2         taster_name  \
0                 Etna               None       Kerin O’Keefe   
1                 None      

In [2]:
data['points'] = pandas.to_numeric(data['points'])
data.dropna(subset=['points', 'price'])
data['variety'].value_counts().nlargest(10)

Pinot Noir                  13272
Chardonnay                  11753
Cabernet Sauvignon           9472
Red Blend                    8946
Bordeaux-style Red Blend     6915
Riesling                     5189
Sauvignon Blanc              4967
Syrah                        4142
Rosé                         3564
Merlot                       3102
Name: variety, dtype: int64

In [3]:
df = data.groupby('variety', as_index=True)[['price','points']].mean()
df1 = df[(df.price < 20) & (df.points >=90)]
print(df1)

                              price  points
variety                                    
Blauburgunder             19.000000    93.0
Caprettone                19.000000    92.0
Kotsifali                 13.000000    92.0
Ondenc                    15.000000    90.0
Roussanne-Grenache Blanc  16.000000    91.0
Shiraz-Malbec             18.666667    90.0
Tinta Cao                 14.000000    90.0


In [4]:
country_df = data.groupby(['country','variety'], as_index=True)[['price','points']].mean()
print(country_df.head())
pandas.DataFrame(country_df).to_csv('statisticByState.csv', sep=',')

                                          price     points
country   variety                                         
Argentina Barbera                     18.000000  85.000000
          Bonarda                     16.628571  86.504762
          Bordeaux-style Red Blend    41.546512  89.820225
          Bordeaux-style White Blend  14.000000  83.000000
          Cabernet Blend              36.250000  88.250000


In [0]:
pandas.DataFrame(data['description']).to_csv('desc.csv', sep='\t')


# Text Analysis

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
word_vectorizer = CountVectorizer(ngram_range=(1,2), analyzer='word')
sh = [line.strip() for line in open("desc.csv", encoding="utf8")]
desc_txt = [word.lower() for word in sh]
sparse_matrix = word_vectorizer.fit_transform(desc_txt)
frequencies = sum(sparse_matrix).data
highFreq = pandas.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
hf = highFreq[highFreq['frequency']>5000]
pandas.DataFrame(hf).to_csv('HighFreq.txt', sep='\t')

In [1]:
stopword = pandas.read_fwf('stopwords.txt')
data['description'] = data['description'].str.lower().str.split()
shirax = data['description'].apply(lambda x: [item for item in str(x).split() if item not in stopword])
#print(shirax.head())
pandas.DataFrame(shirax).to_csv('out.csv', sep='\t')

NameError: ignored

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english',norm = None)
shi = [line.strip() for line in open("out.csv", encoding="utf8")]
texts = [word.lower() for word in shi]
X = vectorizer.fit_transform(texts)
X_vovab = vectorizer.get_feature_names()
X_mat = X.todense()
X_idf = vectorizer.idf_
shiraz_df = pandas.DataFrame({'key':X_vovab,'idf':X_idf})
shira = shiraz_df[shiraz_df['idf'] > 0.4]
pandas.DataFrame(shira).to_csv('Shirazkey.txt', sep='\t')